In [1]:
import os
import re
import pickle
from __future__ import print_function
from time import time

# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [2]:

filepath = 'gcat_grp_cat'
file = open(filepath, 'rb')
group_cat = pickle.load(file)
cnt = 0
for key in group_cat.keys():
    print("key:",key)
    print("val:",group_cat[key])
    cnt += 1
    if cnt > 10:
        break
file.close()


## will create dict from event to category
filepath = 'egroup'
file = open(filepath, 'rb')
event_group = pickle.load(file)
file.close()
"""
filepath = 'gcat'
file = open(filepath, 'rb')
group_cat = pickle.load(file)
file.close()
"""

key: 18497538
val: 2
key: 1671174
val: 4
key: 1187853
val: 2
key: 11870222
val: 0
key: 19513360
val: 1
key: 1826841
val: 1
key: 19587098
val: 0
key: 1703963
val: 4
key: 18489374
val: 4
key: 1368097
val: 4
key: 16621602
val: 1


"\nfilepath = 'gcat'\nfile = open(filepath, 'rb')\ngroup_cat = pickle.load(file)\nfile.close()\n"

In [9]:

event_cat = {}
for event,group in event_group.items():
    #print("event:",event)
    #print("group:",group)
    try:
        event_cat[event] = (group_cat[group])
        if group_cat[group] > 5:
            print(group_cat[group])
        # print("group_cat:",(group_cat[group]))
    except:
        pass

filename = 'event_cat'
outfile = open(filename,'wb')
pickle.dump(event_cat,outfile)
outfile.close()


In [10]:
print("event_cat size:",len(event_cat))
description_dict = {}
event_dict = {}
filepath = 'event_preprocess_description_pickle'
file = open(filepath, 'rb')
event_description = pickle.load(file)
file.close()
for event,description in event_description.items():
    # Remove punctuation
    description = re.sub('[,\.!?]', '', description)
    description = description.lower()
    try:
        category = event_cat[event]
        if category not in description_dict:
            description_dict[category] = []
            event_dict[category] = []
        description_dict[category].append(description)
        event_dict[category].append(event)
    except:
        pass

event_cat size: 442462


In [11]:
n_samples = 2000
n_features = 1000
n_topics = 50
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

category_event_vector_dict = {}
for category,description_list in description_dict.items():
    if category == 24:
        continue
    print("category",category,"len of description_list",len(description_list))    
    print("Loading dataset...")
    t0 = time()
    data_samples = description_list
    print("done in %0.3fs." % (time() - t0))

    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                       max_features=n_features,
                                       stop_words='english')
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data_samples)
    print("done in %0.3fs." % (time() - t0))


    # Fit the NMF model
    print("Fitting the NMF model with tf-idf features, "
          "n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    t0 = time()
    nmf = NMF(n_components=n_topics, random_state=1,
              alpha=.1, l1_ratio=.5).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model:")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    #print_top_words(nmf, tfidf_feature_names, n_top_words)
    count_transform = nmf.transform(tfidf)
    # dump information to that file
    data = event_dict[category]
    

    my_data = {}
    cnt = 0
    for values in data:
        my_vector = count_transform[cnt][:]
        #print("value:",values)
        cnt = cnt +1
        my_vector_l = list(my_vector)
        my_data[values] = my_vector_l
        #print("event:",values,"vector",my_data[values])
    category_event_vector_dict[category] = my_data
    print(len(my_data))
    



category 0 len of description_list 20276
Loading dataset...
done in 0.000s.
Extracting tf-idf features for NMF...
done in 2.911s.
Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
done in 21.633s.

Topics in NMF model:
20276
category 1 len of description_list 6903
Loading dataset...
done in 0.000s.
Extracting tf-idf features for NMF...
done in 0.879s.
Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
done in 3.946s.

Topics in NMF model:
6903
category 2 len of description_list 18799
Loading dataset...
done in 0.000s.
Extracting tf-idf features for NMF...
done in 2.105s.
Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
done in 17.372s.

Topics in NMF model:
18799
category 3 len of description_list 2033
Loading dataset...
done in 0.000s.
Extracting tf-idf features for NMF...
done in 0.283s.
Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
done in 1.361s.

Topi

In [12]:
filename = 'category_event_topic_vector_nnmf'
outfile = open(filename,'wb')
pickle.dump(category_event_vector_dict,outfile)
outfile.close()